In [ ]:
# Copiado desde el canvas, en versión de script
# Puedes ejecutar celda por celda en Binder

# === IMPORTS Y CONFIGURACIÓN ===
from pyomo.environ import *
import pandas as pd
from datetime import datetime, timedelta

horas_diarias = 7.6
rate = 100
severidad = 1.1
rate_lessor = 230
tasa_descuento = 0.11
duracion_reparacion = 275
lim_core = 20000
lim_nocore = 30000
costo_LLP_core = 2_000_000
costo_LLP_nocore = 2_000_000
umbral_sv1 = 9000
umbral_sv2 = 5000
fecha_base = datetime(2025, 3, 15)
escalacion = 0.03

def escalar(valor_base, t_dias):
    return valor_base * (1 + escalacion) ** (t_dias / 365)

def FH(t, h0):
    return h0 + horas_diarias * t

def FC(t, c0):
    return c0 + t

df = pd.read_excel("PSRV optimización.xlsx")
df.columns = df.columns.str.strip()
df["Fecha SV"] = pd.to_datetime(df["Next ESV EGTM 0°"])
df["Dias_limite"] = (df["Fecha SV"] - fecha_base).dt.days
df["TSN"] = df["TSN"].astype(float)
df["CSN"] = df["CSN"].astype(float)

model = ConcreteModel()
model.MOTORES = Set(initialize=df["Original"].tolist())

h0 = dict(zip(df["Original"], df["TSN"]))
c0 = dict(zip(df["Original"], df["CSN"]))
lim = dict(zip(df["Original"], df["Dias_limite"]))

model.t = Var(model.MOTORES, domain=NonNegativeReals, bounds=lambda m, i: (0, lim[i]))
model.Costo_SV1 = Var(model.MOTORES, domain=NonNegativeReals)
model.Costo_SV2 = Var(model.MOTORES, domain=NonNegativeReals)
model.Costo_EOL = Var(model.MOTORES, domain=NonNegativeReals)

def regla_sv1(model, m):
    t = model.t[m]
    fh = FH(t, h0[m])
    fc = FC(t, c0[m])
    rate_t = escalar(rate, t)
    base = rate_t * severidad * fh
    llp_core = escalar(costo_LLP_core, t) if (lim_core - fc) < umbral_sv1 else 0
    llp_nocore = escalar(costo_LLP_nocore, t) if (lim_nocore - fc) < umbral_sv1 else 0
    return model.Costo_SV1[m] == base + llp_core + llp_nocore

model.restric_sv1 = Constraint(model.MOTORES, rule=regla_sv1)

def regla_sv2(model, m):
    t = model.t[m]
    t2 = t + 17200 / horas_diarias
    fh2 = FH(t2, h0[m])
    fc2 = FC(t2, c0[m])
    if (lim_core - fc2) < umbral_sv2 or (lim_nocore - fc2) < umbral_sv2:
        rate_t2 = escalar(rate, t2)
        base = rate_t2 * severidad * fh2
        llp_core = escalar(costo_LLP_core, t2) if (lim_core - fc2) < umbral_sv1 else 0
        llp_nocore = escalar(costo_LLP_nocore, t2) if (lim_nocore - fc2) < umbral_sv1 else 0
        return model.Costo_SV2[m] == base + llp_core + llp_nocore
    else:
        return model.Costo_SV2[m] == 0

model.restric_sv2 = Constraint(model.MOTORES, rule=regla_sv2)

def regla_eol(model, m):
    t = model.t[m]
    fh_sv1 = FH(t, h0[m])
    fh_final = FH(4320, h0[m])
    desgaste = max(0, fh_final - fh_sv1)
    rate_less = escalar(rate_lessor, t)
    return model.Costo_EOL[m] == rate_less * desgaste

model.restric_eol = Constraint(model.MOTORES, rule=regla_eol)

def npv(model):
    return sum(
        model.Costo_SV1[m] / (1 + tasa_descuento) ** (model.t[m] / 365) +
        model.Costo_SV2[m] / (1 + tasa_descuento) ** ((model.t[m] + 17200 / horas_diarias) / 365) +
        model.Costo_EOL[m] / (1 + tasa_descuento) ** (4320 / 365)
        for m in model.MOTORES
    )

model.obj = Objective(rule=npv, sense=minimize)

from pyomo.opt import SolverFactory
solver = SolverFactory("bonmin")
results = solver.solve(model, tee=True)

resultados = []
for m in model.MOTORES:
    t_inicio = int(value(model.t[m]))
    fecha_inicio = fecha_base + timedelta(days=t_inicio)
    fecha_fin = fecha_inicio + timedelta(days=duracion_reparacion)
    fecha_limite = df.loc[df["Original"] == m, "Fecha SV"].values[0]
    dias_anticipacion = (pd.to_datetime(str(fecha_limite)) - fecha_inicio).days
    resultados.append({
        "Original": m,
        "Fecha Inicio Reparación": fecha_inicio.strftime("%Y-%m-%d"),
        "Fecha Fin Reparación": fecha_fin.strftime("%Y-%m-%d"),
        "Días de Anticipación": dias_anticipacion,
        "Costo SV1": round(value(model.Costo_SV1[m]), 2),
        "Costo SV2": round(value(model.Costo_SV2[m]), 2),
        "Costo EOL": round(value(model.Costo_EOL[m]), 2),
    })

res = pd.DataFrame(resultados)
res.to_excel("Resultados_Bonmin.xlsx", index=False)
print("✅ Resultados exportados correctamente.")
